# Titanic survival

Given a dataset of Titanic passengers, predict which ones survived.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plot
%matplotlib inline

#Machine learning
import sklearn as sk

In [2]:
#Read data
train = pd.read_csv("./csv/train.csv")
test = pd.read_csv("./csv/test.csv")
test_class = pd.read_csv("./csv/gender_submission.csv")

#Data preview
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## Preprocess data

We'll select as features
* Passenger class
* Sex
* Age (When age is NaN, we use the mean value of each class as a default value)

In [3]:
def process_nan_ages(data):
    clean_data = (data.dropna())
    survived = data[data.Survived == 1]
    died = data[data.Survived == 0]
    avg_survived = survived.Age.mean()
    avg_died = died.Age.mean()
    data.loc[(data.Survived == 0) & (data.Age.isnull()), 'Age'] = avg_died
    data.loc[(data.Survived == 1) & (data.Age.isnull()), 'Age'] = avg_survived 

In [4]:
#Assing numeric values to sex
train.Sex = train.Sex.replace('male', 0)
train.Sex = train.Sex.replace('female', 1)

#Fix NaN values on age field
process_nan_ages(train)

#Select features
train_features = train[['Pclass', 'Sex', 'Age']]
train_class = train['Survived']

#Preview trainning set
train_features.head(10)

,Pclass,Sex,Age
0,3,0,22.000000
1,1,1,38.000000
2,3,1,26.000000
3,1,1,35.000000
4,3,0,35.000000
5,3,0,30.626179
6,1,0,54.000000
7,3,0,2.000000
8,3,1,27.000000
9,2,1,14.000000


### Preprocess test set

In [5]:
#Join 'survived' values in test
test = pd.merge(test, test_class, how = "left", on = "PassengerId")

#Assing numeric values to sex
test.Sex = test.Sex.replace('male', 0)
test.Sex = test.Sex.replace('female', 1)

#Fix NaN values on age field
process_nan_ages(test)

#Select features
test_features = test[['Pclass', 'Sex', 'Age']]
test_class = test['Survived']

#Preview trainning set
test_features.head()

,Pclass,Sex,Age
0,3,0,34.5
1,3,1,47.0
2,2,0,62.0
3,3,0,27.0
4,3,1,22.0


## Machine learning with scikit-learn

In [6]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors = 10)
knn.fit(train_features, train_class)

test_pred = knn.predict(test_features)
accuracy_score(test_class, test_pred)

0.74401913875598091